# FrozenLake Game - Google Colab Version

Este notebook implementa o jogo FrozenLake usando matplotlib para visualização e ipywidgets para controle interativo. O objetivo é controlar um agente para alcançar o objetivo (célula verde) sem cair em buracos (células azuis).

## Como jogar:
- Use os botões **↑ ↓ ← →** para mover o agente (círculo vermelho)
- Evite os buracos (azuis)
- Alcance o objetivo (verde)
- O jogo reinicia automaticamente ao cair ou vencer

## 1. Importar Bibliotecas Necessárias

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import ipywidgets as widgets
from IPython.display import display, clear_output

print("✓ Bibliotecas importadas com sucesso!")

## 2. Definir o Ambiente FrozenLake

In [ ]:
# Definindo o tamanho do ambiente (tabuleiro 4x4)
n_rows, n_cols = 4, 4

# Configurações de células:
# 0 = caminho livre (branco)
# 1 = buraco/gelo frágil (azul)
# 2 = objetivo/meta (verde)

frozen_lake_map = np.array([
    [0, 0, 0, 0],
    [0, 1, 0, 1],
    [0, 0, 0, 1],
    [0, 1, 0, 2]
])

# Posição inicial do agente (canto superior esquerdo)
agent_position = [0, 0]

# Estatísticas do jogo
game_stats = {
    'moves': 0,
    'status': 'Jogando...',
    'won': False,
    'lost': False
}

print("Mapa FrozenLake criado:")
print("0 = Caminho livre | 1 = Buraco | 2 = Objetivo")
print("\nMapa:")
print(frozen_lake_map)

## 3. Criar Funções de Visualização

In [ ]:
def draw_frozen_lake(agent_pos, ax=None):
    """
    Desenha o tabuleiro do FrozenLake com matplotlib
    
    Parâmetros:
    - agent_pos: posição [linha, coluna] do agente
    - ax: eixo matplotlib para desenhar
    """
    if ax is None:
        fig, ax = plt.subplots(figsize=(6, 6))
    else:
        ax.clear()
    
    # Cores para cada tipo de célula
    colors = {
        0: 'white',      # caminho livre
        1: '#4A90E2',    # buraco (azul)
        2: '#7ED321'     # objetivo (verde)
    }
    
    # Desenhar cada célula do tabuleiro
    for row in range(n_rows):
        for col in range(n_cols):
            cell_value = frozen_lake_map[row, col]
            color = colors[cell_value]
            
            # Desenhar retângulo da célula
            rect = plt.Rectangle((col, n_rows - row - 1), 1, 1, 
                                 linewidth=2, edgecolor='black', 
                                 facecolor=color)
            ax.add_patch(rect)
            
            # Adicionar label para a célula se necessário
            if cell_value == 2:  # Objetivo
                ax.text(col + 0.5, n_rows - row - 1 + 0.5, '★',
                       ha='center', va='center', fontsize=20)
            elif cell_value == 1:  # Buraco
                ax.text(col + 0.5, n_rows - row - 1 + 0.5, '✗',
                       ha='center', va='center', fontsize=16, color='white')
    
    # Desenhar o agente (círculo vermelho)
    agent_x = agent_pos[1] + 0.5
    agent_y = n_rows - agent_pos[0] - 1 + 0.5
    circle = Circle((agent_x, agent_y), 0.2, color='red', zorder=10)
    ax.add_patch(circle)
    
    # Configurar eixos
    ax.set_xlim(0, n_cols)
    ax.set_ylim(0, n_rows)
    ax.set_aspect('equal')
    ax.axis('off')
    
    # Título
    status_color = 'green' if game_stats['won'] else 'red' if game_stats['lost'] else 'black'
    title = f"FrozenLake - Movimentos: {game_stats['moves']} | Status: {game_stats['status']}"
    ax.set_title(title, fontsize=14, fontweight='bold', color=status_color)
    
    return ax

# Criar a figura inicialmente
fig, ax = plt.subplots(figsize=(6, 6))
draw_frozen_lake(agent_position, ax)
plt.tight_layout()
plt.show()

print("✓ Funções de visualização criadas!")

## 4. Implementar Lógica de Movimento

In [ ]:
def move_agent(action):
    """
    Move o agente baseado na ação fornecida
    
    Parâmetros:
    - action: string ('up', 'down', 'left', 'right')
    
    Retorna:
    - True se o movimento foi realizado, False caso contrário
    """
    global agent_position, game_stats
    
    old_position = agent_position.copy()
    
    if action == 'up' and agent_position[0] > 0:
        agent_position[0] -= 1
    elif action == 'down' and agent_position[0] < n_rows - 1:
        agent_position[0] += 1
    elif action == 'left' and agent_position[1] > 0:
        agent_position[1] -= 1
    elif action == 'right' and agent_position[1] < n_cols - 1:
        agent_position[1] += 1
    else:
        return False
    
    # Aumentar contador de movimentos apenas se o movimento foi válido
    if agent_position != old_position:
        game_stats['moves'] += 1
    
    return agent_position != old_position

def check_game_state():
    """
    Verifica o estado do jogo e atualiza as informações
    """
    global game_stats
    
    cell_value = frozen_lake_map[agent_position[0], agent_position[1]]
    
    if cell_value == 1:  # Buraco
        game_stats['status'] = '💥 CAIU EM UM BURACO!'
        game_stats['lost'] = True
        return 'lost'
    elif cell_value == 2:  # Objetivo
        game_stats['status'] = '🎉 OBJETIVO ALCANÇADO!'
        game_stats['won'] = True
        return 'won'
    else:
        game_stats['status'] = 'Jogando...'
        game_stats['won'] = False
        game_stats['lost'] = False
        return 'playing'

def reset_game():
    """
    Reinicia o jogo
    """
    global agent_position, game_stats
    agent_position = [0, 0]
    game_stats = {
        'moves': 0,
        'status': 'Jogando...',
        'won': False,
        'lost': False
    }

print("✓ Funções de movimento implementadas!")

## 5. Executar o Jogo Interativo

Use os botões abaixo para controlar o agente. Clique em **Reiniciar** para começar um novo jogo.

In [ ]:
# Criar a interface do jogo
output = widgets.Output()
game_display = widgets.Output()

# Botões de controle
btn_up = widgets.Button(description='↑', button_style='info', layout=widgets.Layout(width='50px'))
btn_down = widgets.Button(description='↓', button_style='info', layout=widgets.Layout(width='50px'))
btn_left = widgets.Button(description='←', button_style='info', layout=widgets.Layout(width='50px'))
btn_right = widgets.Button(description='→', button_style='info', layout=widgets.Layout(width='50px'))
btn_reset = widgets.Button(description='Reiniciar', button_style='warning', layout=widgets.Layout(width='100px'))

# Layout dos botões
button_layout = widgets.VBox([
    widgets.HBox([widgets.Label(value='Movimento:', layout=widgets.Layout(width='70px')), 
                  widgets.Label(value='')]),
    widgets.HBox([widgets.Label(value=''), btn_up, widgets.Label(value='')],
                layout=widgets.Layout(justify_content='center')),
    widgets.HBox([btn_left, widgets.Label(value=''), btn_up, widgets.Label(value=''), btn_right],
                layout=widgets.Layout(justify_content='center')),
    widgets.HBox([widgets.Label(value=''), btn_down, widgets.Label(value='')],
                layout=widgets.Layout(justify_content='center')),
    widgets.HBox([btn_reset], layout=widgets.Layout(justify_content='center'))
])

def update_display():
    """Atualiza a visualização do jogo"""
    with game_display:
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(6, 6))
        draw_frozen_lake(agent_position, ax)
        plt.tight_layout()
        plt.show()

def on_button_click(action):
    """Callback para cliques nos botões"""
    def handler(button):
        if not game_stats['won'] and not game_stats['lost']:
            move_agent(action)
            state = check_game_state()
            update_display()
            
            if state == 'won':
                with output:
                    print(f"🎉 Parabéns! Você alcançou o objetivo em {game_stats['moves']} movimentos!")
            elif state == 'lost':
                with output:
                    print(f"💥 Você caiu em um buraco! Reinicie para tentar novamente.")
    return handler

def on_reset_click(button):
    """Callback para o botão de reiniciar"""
    reset_game()
    update_display()
    with output:
        clear_output()
        print("🎮 Jogo reiniciado! Use os botões para mover.")

# Conectar callbacks aos botões
btn_up.on_click(on_button_click('up'))
btn_down.on_click(on_button_click('down'))
btn_left.on_click(on_button_click('left'))
btn_right.on_click(on_button_click('right'))
btn_reset.on_click(on_reset_click)

# Exibir interface
print("🎮 Jogo FrozenLake iniciado!")
print("Use os botões para mover o agente (círculo vermelho)")
print("Evite os buracos (azuis) e alcance o objetivo (verde)\n")

update_display()

display(button_layout)
display(output)

## 6. Informações Adicionais

### Sobre o FrozenLake
- **Objetivo**: Mover o agente (círculo vermelho) do canto superior esquerdo até o objetivo (★ verde)
- **Obstáculos**: Evitar os buracos (✗ azuis)
- **Movimentos**: Apenas 4 direções (cima, baixo, esquerda, direita)
- **Limites**: O agente não pode sair do tabuleiro 4x4

### Diferenças da versão Pygame
- ✅ **Visualização com Matplotlib** - Melhor compatibilidade com Colab
- ✅ **Controles com Botões** - Interface intuitiva com iPyWidgets
- ✅ **Exibição em Tempo Real** - Atualização imediata após cada movimento
- ✅ **Compatível com Google Colab** - Funciona sem Pygame

### Como Usar em seu Próprio Notebook
Você pode copiar este código para seus projetos de IA e aprendizado por reforço!